# Elastic engine demo

In [1]:
import datalabframework as dlf
from datalabframework import utils
dlf.project.info()

{'profile': 'default',
 'filename': 'elastic.ipynb',
 'rootpath': '/home/thuc/teko/product-search/dlf-demos_fork/demos/elastic',
 'workdir': '/home/thuc/teko/product-search/dlf-demos_fork/demos/elastic'}

In [2]:
metadata = dlf.params.metadata()
utils.pretty_print(metadata)
# just the resources, some pretty printing
utils.pretty_print(metadata['resources'])

engines:
  spark-local:
    config:
      jobname: default
      master: local[*]
    context: spark
loggers:
  severity: info
  stream:
    enable: true
profile: default
providers:
  elastic_test:
    hostname: 123.31.32.226
    port: 9200
    service: elastic
  local_filesystem:
    format: csv
    path: data
    service: local
resources:
  .keywords:
    path: keywords.csv
    provider: local_filesystem
    read:
      mapping:
        abc:
          drop: true
          fields:
            keyword:
              ignore_above: 256
              type: keyword
          fill: value
          name: cds
          type: text
      options:
        header: true
        inferSchema: true
  .keywords_elk:
    path: search_keywords_dev2
    provider: elastic_test
    write:
      options:
        mappings:
          doc_type: keyword
          properties:
            api_result: integer
            count: integer
            gg: integer
            gg_clicks: double
            gg_impression

In [3]:
import datalabframework as dlf
engine = dlf.engines.get('spark-local')

PYSPARK_SUBMIT_ARGS:  pyspark-shell


In [4]:
# dlf.data.metadata('keywords_elk')

In [5]:
res = engine.read(".keywords_search", _query="laptop i7", _from=0, _size=100)
res.show()

{'size': 100, 'from': 0, 'query': {'function_score': {'query': {'match': {'query_wo_tones': {'query': 'laptop i7', 'operator': 'or', 'fuzziness': 'AUTO'}}}, 'script_score': {'script': {'source': "Math.sqrt(doc['impressions'].value)"}}, 'boost_mode': 'multiply'}}}
Summary: {'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'total_hits': 212, 'max_score': 645.10345}


--- Logging error ---
Traceback (most recent call last):
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 993, in emit
    msg = self.format(record)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 839, in format
    return fmt.format(record)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 579, in format
    s = self.formatMessage(record)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 548, in formatMessage
    return self._style.format(record)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
KeyError: 'dlf_session'
Call stack:
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/thuc

2018-10-12 17:10:31,879 - INFO - 30d76c6 - datalabframework-demos.git - thuc - elastic.ipynb - engine.read - {'format': None, 'service': 'elastic', 'path': '/search_keywords_dev2/keyword/', 'url': 'http://123.31.32.226:9200//search_keywords_dev2/keyword/'}
+--------------------+--------------------+---------+-------+----------+-----+---+--------------+------------------+-----------+--------+---+---------+----------------+----------------+
|                 _id|              _index|   _score|  _type|api_result|count| gg|gg_impressions|       gg_position|impressions|position| pv|pv_result|           query|  query_wo_tones|
+--------------------+--------------------+---------+-------+----------+-----+---+--------------+------------------+-----------+--------+---+---------+----------------+----------------+
|ydocYWYBknddiPsCTaJ4|search_keywords_dev2|645.10345|keyword|     966.0|    7|  1|      253461.0| 4.763130660666666|    33391.0|     1.0|  1|    726.0|          laptop|          laptop|

In [6]:
dlf.project.repository()

{'type': 'git',
 'committer': 'thucnguyen',
 'hash': '30d76c6',
 'commit': '30d76c630af9129c6f63572ba6232825790d6c16',
 'branch': 'master',
 'url': 'git@github.com:ncthuc/datalabframework-demos.git',
 'name': 'datalabframework-demos.git',
 'date': '2018-10-11T10:13:59+07:00',
 'clean': False}

In [7]:
type(res)

pyspark.sql.dataframe.DataFrame

In [8]:
df = engine.read(".keywords")

2018-10-12 17:10:34,884 - INFO - 30d76c6 - datalabframework-demos.git - thuc - elastic.ipynb - engine.read - {'format': 'csv', 'service': 'local', 'path': 'keywords.csv', 'url': 'file:///home/thuc/teko/product-search/dlf-demos_fork/demos/elastic/data/keywords.csv'}


In [9]:
df.show()

+--------------------+--------------------+-----------+--------+--------------+-----------+----------+---------+---+---+-----+
|      query_wo_tones|               query|impressions|position|gg_impressions|gg_position|api_result|pv_result| pv| gg|count|
+--------------------+--------------------+-----------+--------+--------------+-----------+----------+---------+---+---+-----+
|        asus h110m-d|        asus h110m-d|195.8090665|    null|         570.0|3.335087719|     458.0|      0.0|  0|  1|    1|
|           ddr3 1600|           ddr3 1600|      202.0|     1.0|          86.0|        2.5|      30.0|     36.0|  1|  1|    1|
|         epson l1300|         epson l1300|191.3432457|    null|         557.0|5.057450628|       1.0|      1.0|  0|  1|    1|
| intel pentium g4600| intel pentium g4600|166.2659442|    null|         484.0|3.423553719|       1.0|      0.0|  0|  1|    1|
|               j250f|               j250f|220.1993186|    null|         641.0|3.653666147|       1.0|      3.0

In [10]:
utils.pretty_print(dlf.data.metadata("keywords_elk"))

provider:
  hostname: 123.31.32.226
  path: ''
  port: 9200
  service: elastic
resource:
  path: search_keywords_dev2
  provider: elastic_test
  write:
    options:
      mappings:
        doc_type: keyword
        properties:
          api_result: integer
          count: integer
          gg: integer
          gg_clicks: double
          gg_impressions: double
          gg_position: double
          pv: integer
          pv_impressions: double
          pv_position: double
          pv_result: double
          query:
            fields:
              keyword:
                ignore_above: 256
                type: keyword
            type: text
          query_wo_tones:
            fields:
              keyword:
                ignore_above: 256
                type: keyword
            type: text
        properties_bak: "{\n    \"api_result\": {\n        \"type\": \"double\"\n\
          \    },\n    \"count\": {\n        \"type\": \"integer\"\n    },\n    \"\
          gg\": {\n   

In [11]:
engine.write(df, ".keywords_elk", mode='append')

--- Logging error ---
Traceback (most recent call last):
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 993, in emit
    msg = self.format(record)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 839, in format
    return fmt.format(record)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 579, in format
    s = self.formatMessage(record)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 548, in formatMessage
    return self._style.format(record)
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/logging/__init__.py", line 391, in format
    return self._fmt % record.__dict__
KeyError: 'dlf_session'
Call stack:
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/threading.py", line 884, in _bootstrap
    self._bootstrap_inner()
  File "/home/thuc/miniconda3/envs/p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/thuc/min

KeyError: 'format'